In [1]:
wdir='./' ## working dir
gColab=True  ## set this to False if running locally
if(gColab):
    ## Mount google drive to the machine running this nb
    from google.colab import drive
    drive.mount('/content/drive')
    ## must also specify the absolute path to the working dir
    wdir='/content/drive/MyDrive/ColabNtbs/ysNN_PolyN/batch104_zOptim/'

Mounted at /content/drive


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random

import sys
sys.path.append(os.path.abspath(wdir))

import cvxDomainPolyN as cvxn

gseed=99
if(1):
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    tf.random.set_seed(gseed)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

## set off randomness
np.random.seed(gseed)
random.seed(gseed)
os.environ['PYTHONHASHSEED']=str(gseed)
tf.random.set_seed(gseed)


In [3]:
degree=4
nMonoms=int((int(degree/2+1))**2)-1
#cntp=  [ 1. ,   -1.61455962,  3.43721333, -1.7651701,   1.83310375,  6.09711274, -5.91793414,  6.15193756,  9.07579582]
cntp=[0 for k in range(9)]
cntp[0] =  1.0
cntp[1] =  -1.6129362192236751
cntp[2] =  3.454157357252988
cntp[3] =  -1.7653572545874991
cntp[4] =  1.862396791843772
cntp[5] =  6.111839159843277
cntp[6] =  -5.921176129510384
cntp[7] =  6.1850318108821885
cntp[8] =  9.12879232596765
#vonMises =  [ 1. -2.  3. -2.  1.  6. -6.  6.  9.]
dxMax =  [0.0,
8.420482078405513,
60.72447809357019,
100.28299963683395,
123.27000099311208,
78.44972776373547,
72.87225156911263,
125.62917310072585,
246.0112253395461]
cntp=np.array(cntp[1:]).reshape((1,nMonoms))
dxMax=np.array(dxMax[1:]).reshape((1,nMonoms))

In [4]:
def pAct(x):
    return 0.5*((1+tf.math.sign(x))*(x+0.01)+0.01*(1-tf.math.sign(x))*tf.math.exp(100.0*tf.math.minimum(0.0,x)))

nun1=200
model=tf.keras.Sequential(
        [
          tf.keras.layers.Dense(nun1,activation=pAct, input_shape=(8,)),
          tf.keras.layers.Dense(nun1,activation=pAct),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ]
    )
model.load_weights(wdir+'aFitCVXdeg4_12_C_9D4_Weights', skip_mismatch=False, by_name=False, options=None)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               1800      
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 42201 (164.85 KB)
Trainable params: 42201 (164.85 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [67]:
def modelBdry(x,vParam):
    #return model((x.reshape((1,x.shape[0]))-cntp)/dxMax)-0.5
    return model((x.reshape((1,x.shape[0]))-cntp)/dxMax)-0.0865

def modelBdryGrad(x,vParam):
    y=tf.Variable(x.reshape((1,x.shape[0])))
    with tf.GradientTape() as tape:
        tape.watch(y)
        fY=model((y-cntp)/dxMax)
    return tape.gradient(fY,y)


def fCostFunc(va,vParam):
    vb=va.reshape((1,va.shape[0]))
    return 0.5*np.sum(vb*np.matmul(vb,vParam[0]),axis=1)[0]-np.sum(vb*vParam[1],axis=1)[0]

def fCostGrad(va,vParam):
    return np.matmul(va.reshape((1,va.shape[0])),vParam[0])-vParam[1]

In [45]:
degree=4

'''
Any plane stress state can be represented as:

sigma_x = sigma_L*[cos^2(theta)+q*sin^2(theta)]
sigma_y = sigma_L*[sin^2(theta)+q*cos^2(theta)]
sigma_xy = sigma_L*(1-q)*sin(theta)*cos(theta)

When q=0, we recover the classical uniaxial stress state
(where sigma_L is simply the magnitude of the uniaxial tensile stress
exerted at an angle theta from the rolling direction)

The corresponding r-values are defined by the same formula used
for the classical (uniaxial) r-values:

r(q,theta) = epsilon_W/epsilon_Z
where:
epsilon_W = the width strain (rate)
(the width direction being orthogonal to the direction of the highest principal stress component)
epsilon_Z = the thickness strain (rate)

Note that for  in-plane biaxial stress states (sigma_xy = 0 and principal stresses along the x and y-axes),
the above definition of the r-value differs from the one usually employed in the literature.
The latter is defined as the ratio:
r_b = epsilon_y/epsilon_x

Then, if r_b is reported, the relationship with r(q,theta=0) is:
r(q,0) = -r_b/(1+r_b)

Further info on the above parametrization of experimental data in the article:
'Calibration and fast evaluation algorithms for orthotropic homogeneous polynomials'
'''

##########NOTE: stresses must be normalized by the yield stress along RD
data = [  #### AA2090-T3
### uniaxial directional properties (q=0.0)
{'q':0.0,'theta':0.0,'s':'*', 'r':0.2115},
          {'q':0.0,'theta':0.0,'s':'*', 'r':0.2115},
          {'q':0.0,'theta':0.0,'s':'*', 'r':0.2115},
{'q':0.0,'theta':15.0,'s':0.9605, 'r':0.3269},
          {'q':0.0,'theta':15.0,'s':0.9605, 'r':'*'},
          {'q':0.0,'theta':15.0,'s':0.9605, 'r':'*'},
          {'q':0.0,'theta':15.0,'s':0.9605, 'r':'*'},
          {'q':0.0,'theta':15.0,'s':0.9605, 'r':'*'},
          {'q':0.0,'theta':15.0,'s':0.9605, 'r':'*'},
          {'q':0.0,'theta':15.0,'s':0.9605, 'r':'*'},
          {'q':0.0,'theta':15.0,'s':0.9605, 'r':'*'},
          {'q':0.0,'theta':15.0,'s':0.9605, 'r':'*'},
          {'q':0.0,'theta':15.0,'s':0.9605, 'r':'*'},
          {'q':0.0,'theta':15.0,'s':0.9605, 'r':'*'},
{'q':0.0,'theta':30.0,'s':0.9102, 'r':0.6923},
{'q':0.0,'theta':45.0,'s':0.8114, 'r':1.5769},
          {'q':0.0,'theta':45.0,'s':0.8114, 'r':1.5769},
          {'q':0.0,'theta':45.0,'s':0.8114, 'r':1.5769},
          {'q':0.0,'theta':45.0,'s':'*', 'r':1.5769},
          {'q':0.0,'theta':45.0,'s':'*', 'r':1.5769},
          {'q':0.0,'theta':45.0,'s':'*', 'r':1.5769},
          {'q':0.0,'theta':45.0,'s':'*', 'r':1.5769},
          {'q':0.0,'theta':45.0,'s':'*', 'r':1.5769},
{'q':0.0,'theta':60.0,'s':0.8096, 'r':1.0385},
{'q':0.0,'theta':75.0,'s':0.8815, 'r':0.5384},
{'q':0.0,'theta':90.0,'s':0.9102, 'r':0.6923},
          {'q':0.0,'theta':90.0,'s':0.9102, 'r':0.6923},
          {'q':0.0,'theta':90.0,'s':0.9102, 'r':0.6923},
          {'q':0.0,'theta':90.0,'s':'*', 'r':0.6923},
          {'q':0.0,'theta':90.0,'s':'*', 'r':0.6923},
          #{'q':0.0,'theta':90.0,'s':0.9102, 'r':0.6923},
### in-plane biaxial properties
##### balanced-biaxial: q=1.0, theta=0.0
### reported balanced-biaxial r_b value = 0.6700
### use the above formula to get r(q=1,theta=0)=-0.67/1.67
{'q':1.0, 'theta':0.0 ,'s':1.0350 , 'r':-0.4012},
          {'q':1.0, 'theta':0.0 ,'s':1.0350 , 'r':'*'},
### add any other data point...
]

### Note: the overall weight on stress points is 1.0-wR
wR=0.1  ### overall weight on r-values (must be < 1.0)

#r0,s90,r90=data[0]['r'],data[6]['s'],data[6]['r']

vFidx=np.array([0],dtype=int)
vFcoeff=np.ones(1,dtype=np.double)
vData=cvxn.procData(data,wR)
vMonoms=cvxn.vPolyMons(degree)
MM,VV=cvxn.fCostMatrx(vData,vMonoms,vFidx,vFcoeff)

vonMises =  [ 1., -2.,  3., -2.,  1.,  6., -6.,  6.,  9.]
xZro=np.array(vonMises[1:])
if(1):### Take the unconstrained optimum as xZero
    xZro=np.linalg.solve(MM,VV.reshape((VV.shape[1],1))).reshape((VV.shape[1],))
print(xZro)

[-0.67794534  1.41654881 -2.35625137  1.4968451   5.0471218  -1.41734905
  9.25270284 23.51587579]


# Notes    
1) Recall that the cost matrix (and the corresponding unconstrained optimal solution) depends on the overall r-value weight  

2) If X0 is the center (vonMises), then large increments (dLambda) can be used


In [68]:
vfParam,vgParam=[MM,VV],[]
fMin,xSol,minSol=cvxn.constrOptim33CD(xZro,cntp,fCostFunc,fCostGrad,modelBdry,modelBdryGrad,vfParam,vgParam,
                             dLambda=0.3,nIter=10**5)
print('minF = ',fMin)
print('xSol = ', xSol)
print('--------------------Best Sol:')
for key in minSol:
    print(f'{key}: {minSol[key]}')

iter = 3, funcVal = -0.7050034694241678, gradNorm = 0.0009264479308999578, gA = [[-5.5134296e-07]], flagG = True
zero increment and gA =  tf.Tensor([[-5.5134296e-07]], shape=(1, 1), dtype=float32)
Final number of iter = 3
minF =  -0.7050034694241678
xSol =  [-0.68239497  1.42505026 -2.35413402  1.49759785  5.0515799  -1.43662583
  9.23955424 23.454697  ]
--------------------Best Sol:
xMin: [-0.68255733  1.42517055 -2.3540905   1.49743086  5.05154804 -1.43666896
  9.23951811 23.45474659]
fMin: -0.7050031901572686
iter: 1
gFnorm: 0.0011502541299800116
gA: [[-3.6507845e-07]]


In [69]:
!jupyter nbconvert --to html  {wdir+'nb24_aPolyNoptim.ipynb'}

[NbConvertApp] Converting notebook /content/drive/MyDrive/ColabNtbs/ysNN_PolyN/batch104_zOptim/nb24_aPolyNoptim.ipynb to html
[NbConvertApp] Writing 623754 bytes to /content/drive/MyDrive/ColabNtbs/ysNN_PolyN/batch104_zOptim/nb24_aPolyNoptim.html
